In [1]:
from music21 import *
import numpy as np
import os
from collections import Counter
from sklearn.model_selection import train_test_split

/home/tushar/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/tushar/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/tushar/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/tushar/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


In [2]:
def read_midi(file):
    print("Loading music file", file)
    
    notes = []
    notes_to_parse = None
    
    midi = converter.parse(file) #parsing midi files
    
    grp_ins = instrument.partitionByInstrument(midi) #partitioning based on different instrument
    
    for part in grp_ins.parts:
        if 'Piano' in str(part):
            notes_to_parse = part.recurse()
            
            # finding whether particular element is a note
            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch)) #note
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder)) #chord
    return np.array(notes)

In [3]:
path = 'schubert/'
files = [i for i in os.listdir(path) if i.endswith(".mid")]
notes_array = np.array([read_midi(path+i) for i in files])

Loading music file schubert/schub_d960_4.mid
Loading music file schubert/schubert_D935_3.mid
Loading music file schubert/schuim-1.mid
Loading music file schubert/schubert_D850_2.mid
Loading music file schubert/schub_d960_2.mid
Loading music file schubert/schu_143_3.mid
Loading music file schubert/schub_d760_1.mid
Loading music file schubert/schub_d960_1.mid
Loading music file schubert/schubert_D850_3.mid
Loading music file schubert/schumm-4.mid
Loading music file schubert/schubert_D935_1.mid
Loading music file schubert/schumm-5.mid
Loading music file schubert/schubert_D935_2.mid
Loading music file schubert/schuim-2.mid
Loading music file schubert/schuim-3.mid
Loading music file schubert/schubert_D935_4.mid
Loading music file schubert/schuim-4.mid
Loading music file schubert/schumm-6.mid
Loading music file schubert/schub_d960_3.mid
Loading music file schubert/schubert_D850_1.mid
Loading music file schubert/schubert_D850_4.mid
Loading music file schubert/schu_143_2.mid
Loading music file

In [4]:
#converting 2d array to 1d array
notes_ = [element for note_ in notes_array for element in note_]
unique_notes = list(set(notes_))
print(len(unique_notes))

freq = dict(Counter(notes_))
frequent_notes = [note_ for note_, count in freq.items() if count>=50]
print(len(frequent_notes))

304
167


In [5]:
#creating new array frequent music
new_music = []
for notes in notes_array:
    temp = []
    for note_ in notes:
        if note_ in frequent_notes:
            temp.append(note_)
    new_music.append(temp)

new_music = np.array(new_music)

In [6]:
#preparing input and output sequences

no_of_timesteps = 32
x = []
y = []

for note_ in new_music:
    for i in range(0, len(note_) - no_of_timesteps, 1):
        
        input_ = note_[i:i + no_of_timesteps]
        output = note_[i + no_of_timesteps]
        
        x.append(input_)
        y.append(output)
        
x=np.array(x)
y=np.array(y)

In [7]:
# assigning unique integer to every note
unique_x = list(set(x.ravel()))
x_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_x))

#preparing input sequences
x_seq=[]
for i in x:
    temp=[]
    for j in i:
        #assigning unique integer to every note
        temp.append(x_note_to_int[j])
    x_seq.append(temp)
    
x_seq = np.array(x_seq)

In [8]:
#integer sequence for output
unique_y = list(set(y))
y_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_y)) 
y_seq=np.array([y_note_to_int[i] for i in y])

# 80% training and rest 20% for evaluation
x_tr, x_val, y_tr, y_val = train_test_split(x_seq,y_seq,test_size=0.2,random_state=0)

In [9]:
from keras.layers import Conv1D, Dropout, MaxPool1D, Dense, Embedding, GlobalMaxPool1D
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
import keras.backend as k

k.clear_session()
model = Sequential()

#embedding layer
model.add(Embedding(len(unique_x), 100, input_length=32,trainable=True)) 

model.add(Conv1D(64,3, padding='causal',activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))
    
model.add(Conv1D(128,3,activation='relu',dilation_rate=2,padding='causal'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))

model.add(Conv1D(256,3,activation='relu',dilation_rate=4,padding='causal'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))
          
#model.add(Conv1D(256,5,activation='relu'))    
model.add(GlobalMaxPool1D())
    
model.add(Dense(256, activation='relu'))
model.add(Dense(len(unique_y), activation='softmax'))
    
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 32, 100)           16700     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 32, 64)            19264     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 64)            0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 16, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 16, 128)           24704     
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 128)           0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 8, 128)            0         
__________

In [10]:
mc=ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True,verbose=1)

In [11]:
history = model.fit(np.array(x_tr),np.array(y_tr),batch_size=128,epochs=50, validation_data=(np.array(x_val),np.array(y_val)),verbose=1, callbacks=[mc])

Train on 51530 samples, validate on 12883 samples
Epoch 1/50
51530/51530 [==============================] - 39s 766us/step - loss: 4.3404 - val_loss: 4.0236

Epoch 00001: val_loss improved from inf to 4.02363, saving model to best_model.h5
Epoch 2/50
51530/51530 [==============================] - 37s 722us/step - loss: 3.8071 - val_loss: 3.7985

Epoch 00002: val_loss improved from 4.02363 to 3.79854, saving model to best_model.h5
Epoch 3/50
51530/51530 [==============================] - 39s 758us/step - loss: 3.6257 - val_loss: 3.6944

Epoch 00003: val_loss improved from 3.79854 to 3.69437, saving model to best_model.h5
Epoch 4/50
51530/51530 [==============================] - 38s 744us/step - loss: 3.4924 - val_loss: 3.5952

Epoch 00004: val_loss improved from 3.69437 to 3.59523, saving model to best_model.h5
Epoch 5/50
51530/51530 [==============================] - 39s 748us/step - loss: 3.3921 - val_loss: 3.5247

Epoch 00005: val_loss improved from 3.59523 to 3.52475, saving model t

51530/51530 [==============================] - 37s 726us/step - loss: 2.3647 - val_loss: 2.7957

Epoch 00044: val_loss improved from 2.79996 to 2.79572, saving model to best_model.h5
Epoch 45/50
51530/51530 [==============================] - 36s 698us/step - loss: 2.3544 - val_loss: 2.7912

Epoch 00045: val_loss improved from 2.79572 to 2.79124, saving model to best_model.h5
Epoch 46/50
51530/51530 [==============================] - 36s 693us/step - loss: 2.3493 - val_loss: 2.7781

Epoch 00046: val_loss improved from 2.79124 to 2.77813, saving model to best_model.h5
Epoch 47/50
51530/51530 [==============================] - 36s 704us/step - loss: 2.3482 - val_loss: 2.7879

Epoch 00047: val_loss did not improve
Epoch 48/50
51530/51530 [==============================] - 36s 697us/step - loss: 2.3363 - val_loss: 2.7958

Epoch 00048: val_loss did not improve
Epoch 49/50
51530/51530 [==============================] - 36s 708us/step - loss: 2.3298 - val_loss: 2.7778

Epoch 00049: val_loss im

In [12]:
from keras.models import load_model
model = load_model('best_model.h5')

In [13]:
# predicted integer values
import random
ind = np.random.randint(0,len(x_val)-1)

random_music = x_val[ind]

predictions=[]
for i in range(10):

    random_music = random_music.reshape(1,no_of_timesteps)

    prob  = model.predict(random_music)[0]
    y_pred= np.argmax(prob,axis=0)
    predictions.append(y_pred)

    random_music = np.insert(random_music[0],len(random_music[0]),y_pred)
    random_music = random_music[1:]
    
print(predictions)

[115, 50, 35, 149, 35, 115, 149, 35, 35, 35]


In [14]:
# converting integer back into notes

x_int_to_note = dict((number, note_) for number, note_ in enumerate(unique_x)) 
predicted_notes = [x_int_to_note[i] for i in predictions]

In [15]:
# converting the predictions to midi files
def convert_to_midi(prediction_output):
    offset = 0
    output_notes = []
    
    for pattern in prediction_output:
        
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                
                cn=int(current_note)
                new_note = note.Note(cn)
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
                
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
            
        # pattern is a note
        else:
            
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
            
        offset += 1
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='music.mid')

In [16]:
convert_to_midi(predicted_notes)